# Dataloader
The dataloader needs to support the following:  
[x] Data Augmentations.  
[x] Load CIFAR10 dataset through numpy.   
[x] Bag creation procedures.  
    [x] ucc1
    [x] ucc1-4

In [ ]:
import numpy as np
import torch

dataset = np.load("./Data/splitted_cifar10_dataset.npz")

x_test, x_val, x_train, = torch.from_numpy(dataset['x_test']), torch.from_numpy(dataset['x_val']), torch.from_numpy(dataset['x_train'])
y_test, y_train, y_val = torch.from_numpy(dataset['y_test']), torch.from_numpy(dataset['y_train']), torch.from_numpy(dataset['y_val'])

import torch
from torchvision.transforms import v2

transforms = v2.Compose([
    v2.Resize((224,224)),
    v2.ToDtype(torch.float32)
])

from torch.utils.data import Dataset, DataLoader

class CIFAR10(Dataset):
    def __init__(self, image_tensors, image_labels, transform=None, target_transform=None):
        self.image_tensors = image_tensors
        self.image_labels = image_labels
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.image_labels)

    def __getitem__(self, idx):
        # Convert from HWC to CHW
        image = self.image_tensors[idx].permute(2, 1, 0)
        label = self.image_labels[idx].item()
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

train_dataset = CIFAR10(x_train, y_train, transform=transforms)
val_dataset = CIFAR10(x_val, y_val)
test_dataset = CIFAR10(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

train_image, train_label = next(iter(train_loader))
print(train_image.shape)
print(train_label.shape)


In [ ]:
import matplotlib.pyplot as plt

train_image, train_label = next(iter(train_loader))
print(train_image.shape)
print(train_label.shape)

print(train_image[0].shape)
plt.imshow(train_image[0].type(torch.uint8).permute(2, 1, 0))
plt.axis('off')

In [ ]:
import random
from itertools import combinations

def choose_classes(all_labels, ucc_range, generator="random"):

    classes = torch.unique(all_labels).tolist()
    chosen = [] 
    combination_min = 1 # for random generator only
    combination_max = 100 # for random generator only
    
    if generator == "random":
       for ucc in range(ucc_range[0], ucc_range[1] + 1):
            num_combinations = random.randint(combination_min, combination_max)
            for _ in range(num_combinations):
                chosen.append(tuple(random.sample(classes, ucc))) 

    elif generator == "combination":
        for ucc in range(ucc_range[0], ucc_range[1] + 1):
            chosen.extend(combinations(classes, ucc))

    random.shuffle(chosen)

    return chosen

def choose_instances(grouped_indices, classes, generator="even"):

    instances = []
    bag_size = 100

    if generator == "random":
        
        for current, class_index in enumerate(classes):
            # pick a random number of instances from that class
            remainder = bag_size - len(classes) + current + 1
            class_num_instances = random.randint(1, remainder)

            # pick random instances from that class
            for _ in range(class_num_instances):
                idx = random.randrange(0, len(grouped_indices[class_index]))
                instances.append(grouped_indices[class_index][idx])

            bag_size = bag_size - class_num_instances

        
    elif generator == "even":
        # evenly distribute instances across classes
        class_num_instances = [bag_size // len(classes)] * len(classes)
        # distribute the remainder
        remainder = bag_size % len(classes)
        for idx in range(remainder):
            class_num_instances[idx] += 1

        for i, class_index in enumerate(classes):
            for _ in range(class_num_instances[i]):
                idx = random.randrange(0, len(grouped_indices[class_index]))
                print(len(grouped_indices[class_index]), idx)
                instances.append(grouped_indices[class_index][idx])

    return instances
        
        

def bag_loader(dataset, labels, ucc_range, num_bags, bag_size, batch_size=16, shuffle=True):
    # group dataset by labels
    labels = torch.unique(labels)
    grouped_indices = []

    for label in labels:
        indices = []
        for idx, (_, y) in enumerate(dataset):
            if y == label: 
                indices.append(idx)
        grouped_indices.append(indices)

    bags = []
    
    # determine the combinations of classes to pick ahead of time
    picked = choose_classes(labels, ucc_range, generator="combination")
    
    for i in range(num_bags):

        classes = picked[i]
        indices = choose_instances(grouped_indices, classes, generator="random")
        subset = torch.utils.data.Subset(dataset, indices)
        bag = torch.utils.data.DataLoader(subset, batch_size=batch_size, shuffle=shuffle)
        bags.append(bag)

    return bags

bag_loader(val_dataset, y_train, (1,1), 10, 1, 7)

# Model
[x] Google Colab
[] Autoencoder
    [] Encoder
    [] Decoder
    [x] ResNet
    [] Wide ResNet

In [ ]:
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
            

    def forward(self, x):
        residual = x 
        out = self.conv1(x)
        out = self.conv2(out)
        
        if self.downsample:
            residual = self.downsample(x)
            
        out += residual
        out = self.relu(out)
        
        return out


In [ ]:
import torch.nn as nn

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512, num_classes)
        
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.maxpool(x)
        
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x
    


In [ ]:
num_classes = 10
num_epochs = 5
batch_size = 16
learning_rate = 0.01

model = ResNet(ResidualBlock, [3, 4, 6, 3])

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)  


In [ ]:
import gc
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()

    print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item()))
            
    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
    
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))